In [1]:
#!pip install anchorpy

In [2]:
import asyncio
import requests
import json
import logging
from anchorpy import Provider, Wallet, Program
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient

RPC_Endpoint = 'https://api.mainnet-beta.solana.com'
program_id = "cjg3oHmg9uuPsP8D6g29NWvhySJkdYdAo9D25PRbKXJ"

async def get_state(address, program_id, provider_uris):
    """Fetch IDL file from the chain"""
    for index, provider_uri in enumerate(provider_uris):
        try:
            logging.info(f"Trying uri {provider_uri}")
            client = AsyncClient(provider_uri)
            provider = Provider(client, Wallet.dummy())
            program_id_key = PublicKey(program_id)
            
            program = await Program.at(program_id_key, provider)
            logging.debug(program.idl)
            
            address_state = await program.account['State'].fetch(address)
            logging.debug(address_state)
            
            await program.close()

            return address_state
        except Exception as e:
            if index < (len(provider_uris) - 1):
                logging.exception("An exception occurred. Trying another uri")
            else:
                raise e
def getTokenAccountBalance(token_vault, provider_uri):
    data =   {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountBalance",
        "params": [
          str(token_vault),
                {
                "commitment": "finalized",
                "encoding":"jsonParsed"
              } 
        ]
      }
    headers = {"Content-Type": "application/json"}

    ret = requests.post(provider_uri, data=json.dumps(data), headers=headers)
    ret = json.loads(ret.text)
    if 'result' in ret:
        ret = ret['result']['value']["amount"]
    else:
        ret = 0
    return ret

async def getLinkAvailableForPayment(address, program_id, provider_uri):
    try:
        state = await get_state(address, program_id, [provider_uri])
    except:
        return None
    
    countUnpaidRounds, reimbursements = 0, 0
    for oracle in state.oracles.xs:
        numRounds = int(state.config.latest_aggregator_round_id) - int(oracle.from_round_id)
        if numRounds < 0:
            numRounds = 0
        
        countUnpaidRounds += int(numRounds)
        reimbursements += oracle.payment_gjuels
    
    linkBalance = getTokenAccountBalance(state.config.token_vault, provider_uri)
    
    amountDue = (state.config.billing.observation_payment_gjuels)*countUnpaidRounds + reimbursements
    remaining = int(linkBalance) - amountDue
    return remaining

In [3]:
LinkAvailableForPayment = await getLinkAvailableForPayment(
                                "2oyA8ZLwuWeAR5ANyDsiEGueUyDC8jFGFLSixSzT9KtV",
                                 program_id,
                                 RPC_Endpoint, 
                                )
print(LinkAvailableForPayment)

1544825882175
